# Comparing ResNet scratch vs TransferLearning
@hyyoka

><img src="https://drive.google.com/uc?id=1YQkxnNy61Gyi3Gp6ylCKeS72BVruJXr_" width="700" height="500"> 


In [ ]:
!pip install pytorch-ignite

In [ ]:
import torch
from torch import nn, optim
# import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import os
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder 

In [ ]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import EarlyStopping, ModelCheckpoint

# Data 준비

In [ ]:
# gdrive에 mount
from google.colab import drive
drive.mount('/content/gdrive')
# 경로 설정
import os
os.chdir('/content/gdrive/My Drive/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# import shutil
 
# original_dataset_dir = '/content/gdrive/MyDrive/tobigs_week9_plant_leaf'   
# classes_list = os.listdir(original_dataset_dir) 
 
# base_dir = './splitted' 
# os.mkdir(base_dir)
 
# train_dir = os.path.join(base_dir, 'train') 
# os.mkdir(train_dir)
# validation_dir = os.path.join(base_dir, 'val')
# os.mkdir(validation_dir)
# test_dir = os.path.join(base_dir, 'test')
# os.mkdir(test_dir)

# for cls in classes_list:     
#     os.mkdir(os.path.join(train_dir, cls))
#     os.mkdir(os.path.join(validation_dir, cls))
#     os.mkdir(os.path.join(test_dir, cls))

In [ ]:
# print(classes_list)

In [ ]:
# import math
 
# for cls in classes_list[1:]:
#     path = os.path.join(original_dataset_dir, cls)
#     fnames = os.listdir(path)
 
#     train_size = math.floor(len(fnames) * 0.6)
#     validation_size = math.floor(len(fnames) * 0.2)
#     test_size = math.floor(len(fnames) * 0.2)
    
#     train_fnames = fnames[:train_size]
#     print("Train size(",cls,"): ", len(train_fnames))
#     for fname in train_fnames:
#         src = os.path.join(path, fname)
#         dst = os.path.join(os.path.join(train_dir, cls), fname)
#         shutil.copyfile(src, dst)
        
#     validation_fnames = fnames[train_size:(validation_size + train_size)]
#     print("Validation size(",cls,"): ", len(validation_fnames))
#     for fname in validation_fnames:
#         src = os.path.join(path, fname)
#         dst = os.path.join(os.path.join(validation_dir, cls), fname)
#         shutil.copyfile(src, dst)
        
#     test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]

#     print("Test size(",cls,"): ", len(test_fnames))
#     for fname in test_fnames:
#         src = os.path.join(path, fname)
#         dst = os.path.join(os.path.join(test_dir, cls), fname)
#         shutil.copyfile(src, dst)


In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 256 
EPOCH = 30 

In [ ]:
transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()]) 
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base) 
val_dataset = ImageFolder(root='./splitted/val', transform=transform_base)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Baseline Model

In [ ]:
classes_list = ['Apple___healthy', 'Apple___Black_rot', 'Apple___Apple_scab', 'Pepper,_bell___Bacterial_spot', 'Corn___healthy', 'Corn___Northern_Leaf_Blight', 'Potato___healthy', 'Corn___Common_rust', 'Cherry___healthy', 'Strawberry___Leaf_scorch', 'Pepper,_bell___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Potato___Early_blight', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Potato___Late_blight', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Cherry___Powdery_mildew', 'Apple___Cedar_apple_rust', 'Strawberry___healthy']

In [ ]:
num_classes = len(classes_list)
num_classes

21

In [ ]:
class ResBlock(nn.Module):
    def __init__(self,in_ch1,out_ch1, stride1, in_ch2,out_ch2, stride2):
        super(ResBlock, self).__init__()

        self.layers = nn.Sequential(
            nn.BatchNorm2d(num_features=in_ch1),
            nn.ReLU(inplace=True), 
            nn.Conv2d(in_channels=in_ch1, out_channels=out_ch1, kernel_size=3, stride=stride1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=in_ch2),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=in_ch2, out_channels=out_ch2, kernel_size=3, stride=stride2, padding=1, bias=False)
        )

    def forward(self, x):
        out = self.layers(x)
        return x + out

class ResBlock2(nn.Module):
    def __init__(self,in_ch1,out_ch1, stride1, in_ch2,out_ch2, stride2):
        super(ResBlock2, self).__init__()

        self.layers = nn.Sequential(
            nn.BatchNorm2d(num_features=in_ch1),
            nn.ReLU(inplace=True), 
            nn.Conv2d(in_channels=in_ch1, out_channels=out_ch1, kernel_size=3, stride=stride1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=in_ch2),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=in_ch2, out_channels=out_ch2, kernel_size=3, stride=stride2, padding=1, bias=False)
        )
        
        self.revise = nn.Sequential(
            nn.BatchNorm2d(num_features=in_ch1),
            nn.ReLU(inplace=True), 
            nn.Conv2d(in_ch1, out_ch1, 1,2)
        ) 

    def forward(self, x):
        out = self.layers(x)
        x = self.revise(x)
        return x + out


In [ ]:
class IdentityResNet(nn.Module):
    # nblk_stage1: number of blocks in stage 1, nblk_stage2.. 
    def __init__(self, nblk_stage1, nblk_stage2, nblk_stage3, nblk_stage4):
        super(IdentityResNet, self).__init__()

        self.conv = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)

        self.stage1 = nn.Sequential(
            ResBlock(64, 64, 1, 64, 64, 1),
            ResBlock(64, 64, 1, 64, 64, 1)
        )
        self.stage2 = nn.Sequential(
            ResBlock2(64, 128, 2, 128, 128, 1),
            ResBlock(128, 128, 1, 128, 128, 1)
        )
        self.stage3 = nn.Sequential(
            ResBlock2(128, 256, 2, 256, 256, 1),
            ResBlock(256, 256, 1, 256, 256, 1)
        )
        self.stage4 = nn.Sequential(
            ResBlock2(256, 512, 2, 512, 512, 1),
            ResBlock(512, 512, 1, 512, 512, 1)
        )
        self.avgpool = nn.AvgPool2d(kernel_size = 4, stride=6)
        self.fc = nn.Linear(512, num_classes)
    
    def forward(self, x):
        conv = self.conv(x)
        sg1 = self.stage1(conv)
        sg2 = self.stage2(sg1)
        sg3 = self.stage3(sg2)
        sg4 = self.stage4(sg3)
        avg = self.avgpool(sg4)
        print(avg.shape)
        avg = avg.reshape(BATCH_SIZE, 512)
        print(avg.shape)
        out = self.fc(avg)
        print(out.shape)
        return out

In [ ]:
model = IdentityResNet(nblk_stage1=2, nblk_stage2=2,
                     nblk_stage3=2, nblk_stage4=2)
model.to(DEVICE)

optimizer = optim.Adam(model.parameters(), lr=0.001) 
criterion = nn.CrossEntropyLoss() # nn.LogSoftmax + nn.NLLLoss

## Train Model

In [ ]:
# trainer
trainer = create_supervised_trainer(model, optimizer, criterion, device=DEVICE)

# evaluator
metrics = {'accuracy':Accuracy(),'loss':Loss(criterion)}
train_evaluator = create_supervised_evaluator(model, metrics=metrics, device=DEVICE)
train_history = {'accuracy':[],'loss':[]}

val_evaluator = create_supervised_evaluator(model, metrics=metrics, device=DEVICE)
val_history = {'accuracy':[],'loss':[]}

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED) # 한 에폭이 끝날 때마다 실행하게끔 trainer에 추가
def training_log(trainer):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    accuracy = metrics['accuracy']*100
    loss = metrics['loss']
    train_history['accuracy'].append(accuracy)
    train_history['loss'].append(loss)
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}".format(trainer.state.epoch, accuracy, loss))

@trainer.on(Events.EPOCH_COMPLETED)
def validation_log(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    accuracy = metrics['accuracy']*100
    loss = metrics['loss']
    val_history['accuracy'].append(accuracy)
    val_history['loss'].append(loss)
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}".format(trainer.state.epoch, accuracy, loss))
 

### Training!!

In [ ]:
checkpointer = ModelCheckpoint('/content/', 'TOMATO', n_saved=2, create_dir=True, save_as_state_dict=True, require_empty=False)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpointer, {'TOMATO': model})

In [ ]:
trainer.run(train_loader, max_epochs=EPOCH)

In [ ]:
# loading the saved model
def fetch_last_checkpoint_model_filename(model_save_path):
    import os
    checkpoint_files = os.listdir(model_save_path)
    checkpoint_files = [f for f in checkpoint_files if '.pt' in f]
    checkpoint_iter = [
        int(x.split('_')[2].split('.')[0])
        for x in checkpoint_files]
    last_idx = np.array(checkpoint_iter).argmax()
    return os.path.join(model_save_path, checkpoint_files[last_idx])

model.load_state_dict(torch.load(fetch_last_checkpoint_model_filename('/content')))
print("Model Loaded")

### Accuracy plot

In [ ]:
plt.plot(train_history['accuracy'],label="Training Accuracy")
plt.plot(val_history['accuracy'],label="Validation Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
plt.plot(train_history['loss'],label="Training loss")
plt.plot(val_history['loss'],label="Validation loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

# Transfer Learning

In [ ]:
from torchvision import models
 
resnet = models.resnet50(pretrained=True)  
num_ftrs = resnet.fc.in_features   
resnet.fc = nn.Linear(num_ftrs, num_classes 
resnet = resnet.to(DEVICE)
 
criterion = nn.CrossEntropyLoss() 
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)

In [ ]:
# Pre-Trained Model의 일부 Layer Freeze하기 (resnet 기준입니다 !!)
ct = 0 
for child in resnet.children():  
    ct+= 1  
    if ct < 6: 
        for param in child.parameters():
            param.requires_grad = False

In [ ]:
# trainer
f_trainer = create_supervised_trainer(resnet, optimizer_ft, criterion, device=DEVICE)

# evaluator
metrics = {'accuracy':Accuracy(),'loss':Loss(criterion)}
train_evaluator = create_supervised_evaluator(resnet, metrics=metrics, device=DEVICE)
train_history = {'accuracy':[],'loss':[]}

val_evaluator = create_supervised_evaluator(resnet, metrics=metrics, device=DEVICE)
val_history = {'accuracy':[],'loss':[]}

In [ ]:
f_trainer.run(train_loader, max_epochs=EPOCH)